In [1]:
!pip install librosa numpy tensorflow sklearn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import librosa
import numpy as np

# Path to your folder with audio files in Google Drive
folder_path = '/content/drive/MyDrive/Diacwoz/AudioFiles'

# Get all the file paths
audio_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.wav')]

# Check the number of audio files
print(f"Total audio files: {len(audio_files)}")


Total audio files: 138


In [4]:
def extract_mfcc_features(file_path, sample_rate=16000, n_mfcc=40):
    # Load the audio file
    signal, sr = librosa.load(file_path, sr=sample_rate)

    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)

    # Return mean of the MFCC features across time axis
    return np.mean(mfccs.T, axis=0)

# Extract MFCCs for all audio files
X = np.array([extract_mfcc_features(file) for file in audio_files])

# Check the shape of the features
print(f"Feature shape: {X.shape}")


Feature shape: (138, 40)


In [5]:
from sklearn.cluster import KMeans

# Apply k-means clustering to create pseudo-labels
n_clusters = 2  # You can choose any number based on the expected output classes
kmeans = KMeans(n_clusters=n_clusters, random_state=42)

# Fit and predict pseudo-labels
pseudo_labels = kmeans.fit_predict(X)

# Print pseudo-labels
print(pseudo_labels)


[0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1
 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 1
 0 0 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 0 1 1
 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1]


In [23]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model():
    """
    Creates and compiles a CNN model.
    """
    # Reshape input for CNN: (samples, height, width, channels)
    # Original: X_reshaped = X.reshape(X.shape[0], X.shape[1], 1, 1)
    # Reshape to (samples, time steps, features, channels)
    X_reshaped = X.reshape(X.shape[0], X.shape[1], 1, 1)

    # Define a CNN model
    model = models.Sequential()

    # 1st Conv Layer
    # Original: model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], 1, 1)))
    # Adjusted kernel size and stride
    model.add(layers.Conv2D(32, (3, 1), activation='relu', input_shape=(X.shape[1], 1, 1), strides=(1,1)))
    # Changed pool size to (2,1) to avoid reducing the height dimension too much
    model.add(layers.MaxPooling2D((2, 1)))
    model.add(layers.Dropout(0.3))

    # 2nd Conv Layer
    # Original: model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    # Adjusted kernel size and stride
    model.add(layers.Conv2D(64, (3, 1), activation='relu', strides=(1,1)))
    # Changed pool size to (2,1) to avoid reducing the height dimension too much
    model.add(layers.MaxPooling2D((2, 1)))
    model.add(layers.Dropout(0.3))

    # Flatten and Fully Connected Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.3))

    # Output Layer
    model.add(layers.Dense(n_clusters, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model # Return the compiled model

In [24]:
# Define ranges for epochs and batch sizes
epoch_values = [50, 100, 150]  # Experiment with these values
batch_sizes = [16, 32, 64]      # Experiment with these values

In [25]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)  # Adjust patience as needed

In [26]:
best_accuracy = 0
best_params = (0, 0)

for epochs in epoch_values:
    for batch_size in batch_sizes:
        print(f"Training with epochs: {epochs} and batch size: {batch_size}")

        # Create a new instance of the model for each training run
        model = create_model()  # Call the function to create and compile the model

        # Train the model
        history = model.fit(X_reshaped, pseudo_labels,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_split=0.2,
                            callbacks=[early_stopping])

        # Evaluate the model
        loss, accuracy = model.evaluate(X_reshaped, pseudo_labels)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")

        # Check if this is the best accuracy so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = (epochs, batch_size)

print(f"Best Accuracy: {best_accuracy * 100:.2f}% with epochs: {best_params[0]} and batch size: {best_params[1]}")

Training with epochs: 50 and batch size: 16
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - accuracy: 0.6128 - loss: 5.1572 - val_accuracy: 0.2143 - val_loss: 1.3899
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6200 - loss: 2.8666 - val_accuracy: 0.7857 - val_loss: 0.5031
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5984 - loss: 3.1476 - val_accuracy: 0.7857 - val_loss: 0.6115
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6573 - loss: 1.6428 - val_accuracy: 0.7500 - val_loss: 0.5301
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6633 - loss: 1.7726 - val_accuracy: 0.8214 - val_loss: 0.4969
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6479 - loss: 1.5482 - val_accuracy: 0.7857 - val_loss: 0.5437
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.5747 - loss: 1.4974 - val_accuracy: 0.8214 - val_loss: 0.5084
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7305 - loss: 0.8295 - val_accuracy: 0.7500 - val_loss: 0.5240
Epoch 9/50


In [28]:
# Train the model
history = model.fit(X_reshaped, pseudo_labels, epochs=150, batch_size=16, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_reshaped, pseudo_labels)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Epoch 1/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9045 - loss: 0.2414 - val_accuracy: 0.6071 - val_loss: 0.8238
Epoch 2/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8190 - loss: 0.3801 - val_accuracy: 0.6071 - val_loss: 0.8258
Epoch 3/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9068 - loss: 0.2657 - val_accuracy: 0.6071 - val_loss: 0.8709
Epoch 4/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9229 - loss: 0.2661 - val_accuracy: 0.5714 - val_loss: 1.0525
Epoch 5/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8552 - loss: 0.3375 - val_accuracy: 0.5714 - val_loss: 0.9775
Epoch 6/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8900 - loss: 0.3154 - val_accuracy: 0.5714 - val_loss: 0.9704
Epoch 7/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9001 - loss: 0.2818 - val_accuracy: 0.6071 - val_loss: 0.9536
Epoch 8/150
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9195 - loss: 0.3076 - val_accuracy: 0.5714 - val_loss: 1

In [31]:
def predict_on_new_audio(model, file_path):
    # Extract MFCC features from the new audio
    mfcc_features = extract_mfcc_features(file_path)

    # Reshape for CNN input
    mfcc_features_reshaped = mfcc_features.reshape(1, mfcc_features.shape[0], 1, 1)

    # Make a prediction
    prediction = model.predict(mfcc_features_reshaped)
    predicted_label = np.argmax(prediction, axis=1)

    return predicted_label[0]

# Test the model on a new audio file
new_audio_file = '/content/drive/MyDrive/Diacwoz/AudioFiles/300_AUDIO.wav'
predicted_label = predict_on_new_audio(model, new_audio_file)
print(f"Predicted Label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Predicted Label: 0
